Script to investigate the members found at a FPR@$\beta$ of single instance of a attack $a$ v.s. the members found at the same FPR@$\beta$ of multi instance of the same attack $a$.

In [ ]:
import os
import numpy as np
from typing import List, Tuple, Dict
import itertools
from copy import deepcopy
import sys
from pandas import DataFrame
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

sys.path.append("../../../")
sys.path.append("../../")
sys.path.append("../")
from miae.eval_methods.experiment import ExperimentSet, TargetDataset
from miae.eval_methods.prediction import Predictions, HardPreds, plot_roc_hard_preds, get_fpr_tpr_hard_label

sns.set_context("paper")
# set fontsize
plt.rc('xtick', labelsize=7)
plt.rc('ytick', labelsize=7)
plt.rc('legend', fontsize=7)
plt.rc('font', size=7)       
plt.rc('axes', titlesize=8)    
plt.rc('axes', labelsize=8)


plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

mia_name_mapping = {"losstraj": "losstraj", "shokri": "Class-NN", "yeom": "LOSS", "lira": "LIRA", "aug": "aug", "calibration": "calibrated-loss", "reference": "reference"}
mia_color_mapping = {"losstraj": '#1f77b4', "shokri": '#ff7f0e', "yeom": '#2ca02c', "lira": '#d62728', "aug": '#9467bd', "calibration": '#8c564b', "reference": '#e377c2'}


In [ ]:
# parameters:
attack = "reference" # could be "reference", "lira", "shokri", "calibration"
dataset = "cifar10" # could be "cifar10", "cifar100", "cinic10"
base_fpr = 0.01 # the base fpr for each instance
seeds = [0, 1, 2, 3, 4, 5]
model = "resnet56"
ensemble_method = "intersection"
datapath = "home/data/wangz56/miae_experiment_aug_more_target_data"
save_path = f'{datapath}/multi_instance_vs_single_instance'
if not os.path.exists(save_path):
    os.makedirs(save_path)

1. Load the experiment set and dataset

In [ ]:
target_dataset = TargetDataset.from_dir(dataset, f"{datapath}/target/{dataset}")
experiment = ExperimentSet.from_dir(target_dataset, [dataset], datapath, seeds, model)

2. Load the the predictions of `seeds`, adjust them to `base_fpr`, and ensemble them with `ensemble_method`.

3. Calculate the FPR after ensemble (this FPR would be the $\beta$ mentioned above), and pick the same attack with seed 0 (single-instance), to adjust the FPR to the same value $\beta$.

4. Plot Venn Diagram to compare the members found at the same FPR@$\beta$ of single instance of a attack $a$ v.s. the members found at the same FPR@$\beta$ of multi instance of the same attack $a$.